In [28]:
import sionna
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, Camera
import numpy as np
import random
import csv
import os



In [29]:
"""

Mapy
1. losowanie map

"""
def load_random_scene():
    available_scenes = [sionna.rt.scene.munich]


    selected_scene = random.choice(available_scenes)
    
    
    scene = load_scene(selected_scene)
    return scene
    








In [30]:
'''

Scena i kamera

'''

def add_camera(name):
    my_cam = Camera(name, position=[0,0,1000], look_at=scene.center)
    scene.add(my_cam)

def view_scene(scene, name, resolution = [1000,1000]):
    scene.render(name, resolution=resolution, num_samples=512) 
    


In [31]:
'''

Ustawienia nadajnikow i odbiornika

1.Parametry tx i rx
2.Rozmieszczenie tx i rx 
'''
def position(start_x,start_y):
    size = scene.size


    x = random.uniform(start_x, start_x+(size[0].numpy()/3))  
    y = random.uniform(start_y, start_y+(size[1].numpy()/3))  
    z = random.uniform(0, (size[2].numpy())) 
    wyniki= [x, y, z]  
    return wyniki


def position_rx():
    size = scene.size


    x = random.uniform(-(size[0].numpy()/3),(size[0].numpy()/3))  
    y = random.uniform(-(size[1].numpy()/3),(size[1].numpy()/3))  
    z = random.uniform(0, (size[2].numpy())) 
    wyniki= [x, y, z]  
    return wyniki


def set_tx(name,start_x,start_y):
    scene.tx_array = PlanarArray(num_rows=8,
                          num_cols=2,
                          vertical_spacing=0.7,
                          horizontal_spacing=0.5,
                          pattern="tr38901",
                          polarization="VH")
    
    tx = Transmitter(name=name,
                 position=position(start_x,start_y))
    
    scene.add(tx)

def set_rx():
    scene.rx_array = PlanarArray(num_rows=4,
                             num_cols=2,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="dipole",
                             polarization="cross")
    
    
    rx = Receiver(name="rx",
              position=position_rx(),
              orientation=position_rx())


    scene.add(rx)
    


In [32]:
'''

liczenie parametrow na podstawie 
1. dystans

'''
def cal_distance():
    src = paths.sources.numpy()[0]
    trg = paths.targets.numpy()[0]
    
    
    distance = np.sqrt((src[0]- trg[0])**2
                        + (src[1]- trg[1])**2
                        + (src[2]- trg[2])**2)
    return distance

def cal_rtt(dis):
    rtt = (2 * dis) /  (3*10)**8
    return rtt

def cal_doa():
    src = paths.sources.numpy()[0]
    trg = paths.targets.numpy()[0]
    dy = (trg[1] - src[1])
    dx = trg[0] - src[0]
    doa = np.arctan2(dy,dx )
    return doa


def cal_los_nlos(path , distance):
    if path == 0:  # LOS path
        if distance <= 100:
            return 1
        elif distance <= 200:
            return 2
        elif distance <= 400:
            return 3
        elif distance <= 700:
            return 4
        else:
            return 5
    else:  
        return 6
        
    
    

def calculate_param(scene,a, tau, paths):
    # print(paths.types.numpy())
    # print(a.shape)
    if not paths or len(paths.types.numpy()) == 0:
        print("pusta")
    else:
        for i in range(len(paths.types.numpy()[0])):
            # if i >= np.abs(a).shape[5]:
            #     print("blad przy i")
            #     continue
            path = paths.types.numpy()[0]
            value_list = []
            dis = cal_distance()
            value_list.append(cal_los_nlos(path[i],dis))
            value_list.append(dis)
            value_list.append(np.abs(a)[0,0,0,0,0,i,0])
            value_list.append(tau[0,0,0,i].numpy()/1e-9)
            # value_list.append()
            value_list.append(cal_doa())
            value_list.append(cal_rtt(dis))
            save_to_csv("26_2_Test.csv", value_list)
            
        

In [33]:
'''

zapisywanie wartosci 

'''
def save_to_csv(filename, value_list):
    
    
    headers = ["LOS/NLOS", "Distance", "Amplitude", "Tau", "DOA", "RTT"]

    
    file_exists = os.path.isfile(filename)

  
    with open(filename, mode='a', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)


        if not file_exists:
            writer.writerow(headers)

     
        writer.writerow(value_list)







In [35]:

index = 0
while index < 1000000:
    index += 1
     
    scene = load_random_scene()
    size = scene.size
    # print(size)
    
    add_camera("cam_1")
    
    
    
    x = size[0].numpy()/3
    y = size[1].numpy()/3
    
    set_rx()
    set_tx("tx1",-1.3*x,-1.3*y)
    paths = scene.compute_paths(max_depth=5,
                                num_samples=1e6)
    a, tau = paths.cir()
    calculate_param(scene, a, tau, paths)
    scene.remove("tx1")
    set_tx("tx2",-1.3*x,-0.3*y)
    paths = scene.compute_paths(max_depth=5,
                                num_samples=1e6)
    a, tau = paths.cir()
    calculate_param(scene, a, tau, paths)
    scene.remove("tx2")
    set_tx("tx3",-1.3*x,0.3*y)
    paths = scene.compute_paths(max_depth=5,
                                num_samples=1e6)
    a, tau = paths.cir()
    calculate_param(scene, a, tau, paths)
    scene.remove("tx3")
    set_tx("tx4",-0.3*x,-1.3*y)
    paths = scene.compute_paths(max_depth=5,
                                num_samples=1e6)
    a, tau = paths.cir()
    calculate_param(scene, a, tau, paths)
    scene.remove("tx4")
    set_tx("tx5",-0.3*x,-0.3*y)
    paths = scene.compute_paths(max_depth=5,
                                num_samples=1e6)
    a, tau = paths.cir()
    calculate_param(scene, a, tau, paths)
    scene.remove("tx5")
    set_tx("tx6",-0.3*x,0.3*y)
    paths = scene.compute_paths(max_depth=5,
                                num_samples=1e6)
    a, tau = paths.cir()
    calculate_param(scene, a, tau, paths)
    scene.remove("tx6")
    set_tx("tx7",0.3*x,-1.3*y)
    paths = scene.compute_paths(max_depth=5,
                                num_samples=1e6)
    a, tau = paths.cir()
    calculate_param(scene, a, tau, paths)
    scene.remove("tx7")
    set_tx("tx8",0.3*x,-0.3*y)
    paths = scene.compute_paths(max_depth=5,
                                num_samples=1e6)
    a, tau = paths.cir()
    calculate_param(scene, a, tau, paths)
    scene.remove("tx8")
    set_tx("tx9",0.3*x,0.3*y)
    paths = scene.compute_paths(max_depth=5,
                                num_samples=1e6)
    a, tau = paths.cir()
    calculate_param(scene, a, tau, paths)
    scene.remove("tx9")
    print("petla nr", index)
    
    



petla nr 1
petla nr 2
petla nr 3
petla nr 4
petla nr 5
petla nr 6
petla nr 7
petla nr 8
petla nr 9
petla nr 10
petla nr 11
petla nr 12
petla nr 13
petla nr 14
petla nr 15
petla nr 16
petla nr 17
petla nr 18
petla nr 19
petla nr 20
petla nr 21
petla nr 22
petla nr 23
petla nr 24
petla nr 25
petla nr 26
petla nr 27
petla nr 28
petla nr 29
petla nr 30
petla nr 31
petla nr 32
petla nr 33
petla nr 34
petla nr 35
petla nr 36
petla nr 37
petla nr 38
petla nr 39
petla nr 40
petla nr 41
petla nr 42
petla nr 43
petla nr 44
petla nr 45
petla nr 46
petla nr 47
petla nr 48
petla nr 49
petla nr 50
petla nr 51
petla nr 52
petla nr 53
petla nr 54
petla nr 55
petla nr 56
petla nr 57
petla nr 58
petla nr 59
petla nr 60
petla nr 61
petla nr 62
petla nr 63
petla nr 64
petla nr 65
petla nr 66
petla nr 67
petla nr 68
petla nr 69
petla nr 70
petla nr 71
petla nr 72
petla nr 73
petla nr 74
petla nr 75
petla nr 76
petla nr 77
petla nr 78
petla nr 79
petla nr 80
petla nr 81
petla nr 82
petla nr 83
petla nr 84
p


KeyboardInterrupt

